In [1]:
!pip install azure-storage-blob

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.2/394.2 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.9/192.9 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [2]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

def list_blobs_in_container(connection_string, container_name):
    try:
        blob_service_client = BlobServiceClient.from_connection_string(connection_string)
        container_client = blob_service_client.get_container_client(container_name)
        blob_list = container_client.list_blobs()
        for blob in blob_list:
            print(blob.name)
    except Exception as e:
        print(e)

# Usage
connection_string = 'DefaultEndpointsProtocol=https;AccountName=lemonestorageac;AccountKey=x+96vh2uIcIu45ui9WHuh7U1ld0WyusGkdYieIt9rid1toceDN+OyXsT4ZlXq09iJqstCIhSwslx+AStuBZ3PA==;EndpointSuffix=core.windows.net'  # Replace with your connection string
container_name = 'cse-cic-ids2018'  # Replace with your container name
list_blobs_in_container(connection_string, container_name)

Server failed to authenticate the request. Make sure the value of Authorization header is formed correctly including the signature.
RequestId:d20d4e82-801e-005b-40e7-4a7c3c000000
Time:2024-01-19T14:58:43.3305408Z
ErrorCode:AuthenticationFailed
authenticationerrordetail:The MAC signature found in the HTTP request 'FVsq3Npff0n3OxohBl3O9f2scPBbhti+jea2dUOjQhM=' is not the same as any computed signature. Server used following string to sign: 'GET











x-ms-client-request-id:3c8a9b98-b6db-11ee-8fd3-0242ac1c000c
x-ms-date:Fri, 19 Jan 2024 14:58:42 GMT
x-ms-version:2023-11-03
/lemonestorageac/cse-cic-ids2018
comp:list
restype:container'.
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>AuthenticationFailed</Code><Message>Server failed to authenticate the request. Make sure the value of Authorization header is formed correctly including the signature.
RequestId:d20d4e82-801e-005b-40e7-4a7c3c000000
Time:2024-01-19T14:58:43.3305408Z</Message><AuthenticationErrorDetail>The MAC si

In [ ]:
import os
import zipfile

def unzip_blob_files(connection_string, container_name):
    try:
        blob_service_client = BlobServiceClient.from_connection_string(connection_string)
        container_client = blob_service_client.get_container_client(container_name)
        blob_list = container_client.list_blobs()

        for blob in blob_list:
            if blob.name.endswith('.zip'):
                blob_client = container_client.get_blob_client(blob)
                local_zip_file = blob.name.split('/')[-1]

                # Downloading the zip file
                with open(local_zip_file, "wb") as download_file:
                    download_file.write(blob_client.download_blob().readall())

                # Unzipping the file locally
                with zipfile.ZipFile(local_zip_file, 'r') as zip_ref:
                    zip_ref.extractall("unzipped_files")

                # Uploading unzipped files back to the container
                for root, dirs, files in os.walk("unzipped_files"):
                    for file in files:
                        file_path_on_blob = os.path.join(blob.name.replace('.zip', ''), file)
                        file_path_local = os.path.join(root, file)
                        blob_client = container_client.get_blob_client(file_path_on_blob)
                        with open(file_path_local, "rb") as data:
                            blob_client.upload_blob(data, overwrite=True)
                        os.remove(file_path_local)  # Deleting the file after upload

                # Cleaning up: Deleting the downloaded zip file and the unzipped files directory
                os.remove(local_zip_file)
                os.rmdir("unzipped_files")
                print(f'Processed {blob.name}')

    except Exception as e:
        print(e)

In [ ]:
# Usage
unzip_blob_files(connection_string, container_name)


Server failed to authenticate the request. Make sure the value of Authorization header is formed correctly including the signature.
RequestId:53f05938-101e-0004-1275-49c800000000
Time:2024-01-17T18:48:51.5904261Z
ErrorCode:AuthenticationFailed
authenticationerrordetail:The MAC signature found in the HTTP request 'ob6CbfojWnjQQqEqk+6mX2dirx56XEquclBMlP7+9lo=' is not the same as any computed signature. Server used following string to sign: 'GET











x-ms-client-request-id:0e7e9e0c-b569-11ee-b3a9-0242ac1c000c
x-ms-date:Wed, 17 Jan 2024 18:48:51 GMT
x-ms-version:2023-11-03
/lemonestorageac/cse-cic-ids2018
comp:list
restype:container'.
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>AuthenticationFailed</Code><Message>Server failed to authenticate the request. Make sure the value of Authorization header is formed correctly including the signature.
RequestId:53f05938-101e-0004-1275-49c800000000
Time:2024-01-17T18:48:51.5904261Z</Message><AuthenticationErrorDetail>The MAC si

In [ ]:
  print(unzip_blob_files)

<function unzip_blob_files at 0x7b2b03045000>


In [ ]:
for root, dirs, files in os.walk("unzipped_files"):
    for file in files:
        local_file_path = os.path.join(root, file)
        os.remove(local_file_path)  # Deleting the file after upload

In [ ]:
import os
os.rmdir("unzipped_files")

FileNotFoundError: [Errno 2] No such file or directory: 'unzipped_files'

In [3]:
!pip install python-evtx

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 8.4 MB/s eta 0:00:00
  Created wheel for hexdump: filename=hexdump-3.3-py3-none-any.whl size=8894 sha256=48d3f4881b2e83b833bde0f4514f26845f63e78a2caeb653e0099dec58d38a28
  Stored in directory: /root/.cache/pip/wheels/26/28/f7/f47d7ecd9ae44c4457e72c8bb617ef18ab332ee2b2a1047e87
Successfully built hexdump
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.1.1
    Uninstalling pyparsing-3.1.1:
      Successfully uninstalled pyparsing-3.1.1
  Attempting uninstall: more-itertools
    Found existing installation: more-itertools 10.1.0
    Uninstalling more-itertools-10.1.0:
      Successfully uninstalled more-itertools-10.1.0
  Attempting uninstall: zipp
    Found existing installation: zipp 3.17.0
    Uninstalling zipp-3.17.0:
      Successfully uninstalled zipp-3.17.0


In [4]:
import Evtx.Evtx as evtx
import Evtx.Views as e_views

def display_evtx_file_contents(file_path):
    with open(file_path, 'r') as f:
        with evtx.Evtx(f) as log:
            for record in log.records():
                print(e_views.XML_RECORD_VIEW(record.xml()))


In [5]:
from azure.storage.blob import BlobServiceClient
import Evtx.Evtx as evtx
import Evtx.Views as e_views

# Function to download a file from Azure Blob Storage
def download_from_azure(storage_account_connection_string, container_name, blob_name, local_path):
    blob_service_client = BlobServiceClient.from_connection_string(storage_account_connection_string)
    container_client = blob_service_client.get_container_client(container_name)
    blob_client = container_client.get_blob_client(blob_name)
    with open(local_path, "wb") as download_file:
        download_file.write(blob_client.download_blob().readall())

# Function to display the contents of an EVTX file
def display_evtx_file_contents(file_path):
    with evtx.Evtx(file_path) as log:
        for record in log.records():
            print(record.xml())

# User-provided Azure Storage credentials and details
azure_storage_account_name = 'lemonestorageac'  # Replace with your Azure Storage account name
azure_storage_account_key = 'oCEoKyxTbTTlhupoTzWqVfeT2v5vecEo54r0PRHnG6jKCH4H8HnYtCtm0rFrGPDa9aLa9ieVpLxC+AStlakRDw=='   # Replace with your Azure Storage account key
container_name = 'cse-cic-ids2018'    # Replace with your Azure Storage container name
blob_name = 'lemonecsecicids2018out/Original Network Traffic and Log data/Friday-02-03-2018/logs/DESKTOP-AN3U28N-172.31.64.115.evtx'
azure_storage_connection_string ="DefaultEndpointsProtocol=https;AccountName=lemonestorageac;AccountKey=oCEoKyxTbTTlhupoTzWqVfeT2v5vecEo54r0PRHnG6jKCH4H8HnYtCtm0rFrGPDa9aLa9ieVpLxC+AStlakRDw==;EndpointSuffix=core.windows.net"
# Local path where the file will be downloaded
local_file_path = 'local_file.evtx'

# Download the .evtx file from Azure Blob Storage
download_from_azure(azure_storage_connection_string, container_name, blob_name, local_file_path)

# Display the contents of the .evtx file
display_evtx_file_contents(local_file_path)


Streaming output truncated to the last 5000 lines.
<Level>3</Level>
<Task>414</Task>
<Opcode>0</Opcode>
<Keywords>0x4000000000000000</Keywords>
<TimeCreated SystemTime="2018-02-27 12:18:04.778788"></TimeCreated>
<EventRecordID>3298</EventRecordID>
<Correlation ActivityID="" RelatedActivityID=""></Correlation>
<Execution ProcessID="496" ThreadID="1124"></Execution>
<Channel>System</Channel>
<Computer>DESKTOP-AN3U28N</Computer>
<Security UserID="S-1-5-18"></Security>
</System>
<EventData Name="TaskMisconfigured"><Data Name="TaskName">NT TASK\OneDrive Standalone Update Task-S-1-5-21-913095808-1976597276-3100954228-1002</Data>
<Data Name="Parameter">%localappdata%\Microsoft\OneDrive\OneDriveStandaloneUpdater.exe</Data>
</EventData>
</Event>

<Event xmlns="http://schemas.microsoft.com/win/2004/08/events/event"><System><Provider Name="Service Control Manager" Guid="{555908d1-a6d7-4695-8e1e-26931d2012f4}" EventSourceName="Service Control Manager"></Provider>
<EventID Qualifiers="16384">7045</

In [6]:
from azure.storage.blob import BlobServiceClient
import Evtx.Evtx as evtx
import Evtx.Views as e_views

# Function to download a file from Azure Blob Storage
def download_from_azure(storage_account_connection_string, container_name, blob_name, local_path):
    blob_service_client = BlobServiceClient.from_connection_string(storage_account_connection_string)
    container_client = blob_service_client.get_container_client(container_name)
    blob_client = container_client.get_blob_client(blob_name)
    with open(local_path, "wb") as download_file:
        download_file.write(blob_client.download_blob().readall())

# Function to display the contents of an EVTX file
def display_evtx_file_contents(file_path):
    with evtx.Evtx(file_path) as log:
        for record in log.records():
            print(record.xml())

# User-provided Azure Storage credentials and details
azure_storage_account_name = 'lemonestorageac'  # Replace with your Azure Storage account name
azure_storage_account_key = 'oCEoKyxTbTTlhupoTzWqVfeT2v5vecEo54r0PRHnG6jKCH4H8HnYtCtm0rFrGPDa9aLa9ieVpLxC+AStlakRDw=='   # Replace with your Azure Storage account key
container_name = 'cse-cic-ids2018'    # Replace with your Azure Storage container name
blob_name = 'lemonecsecicids2018out/Original Network Traffic and Log data/Friday-02-03-2018/logs/DESKTOP-AN3U28N-172.31.64.118.evtx'
azure_storage_connection_string ="DefaultEndpointsProtocol=https;AccountName=lemonestorageac;AccountKey=oCEoKyxTbTTlhupoTzWqVfeT2v5vecEo54r0PRHnG6jKCH4H8HnYtCtm0rFrGPDa9aLa9ieVpLxC+AStlakRDw==;EndpointSuffix=core.windows.net"
# Local path where the file will be downloaded
local_file_path = 'local_file(1).evtx'

# Download the .evtx file from Azure Blob Storage
download_from_azure(azure_storage_connection_string, container_name, blob_name, local_file_path)

# Display the contents of the .evtx file
display_evtx_file_contents(local_file_path)


Streaming output truncated to the last 5000 lines.
</EventData>
</Event>

<Event xmlns="http://schemas.microsoft.com/win/2004/08/events/event"><System><Provider Name="Microsoft-Windows-Kernel-Boot" Guid="{15ca44ff-4d7a-4baa-bba5-0998955e531e}"></Provider>
<EventID Qualifiers="">18</EventID>
<Version>0</Version>
<Level>4</Level>
<Task>0</Task>
<Opcode>0</Opcode>
<Keywords>0x8000000000000000</Keywords>
<TimeCreated SystemTime="2018-02-27 12:18:04.682817"></TimeCreated>
<EventRecordID>3255</EventRecordID>
<Correlation ActivityID="" RelatedActivityID=""></Correlation>
<Execution ProcessID="4" ThreadID="8"></Execution>
<Channel>System</Channel>
<Computer>DESKTOP-AN3U28N</Computer>
<Security UserID="S-1-5-18"></Security>
</System>
<EventData><Data Name="EntryCount">1</Data>
</EventData>
</Event>

<Event xmlns="http://schemas.microsoft.com/win/2004/08/events/event"><System><Provider Name="Microsoft-Windows-Kernel-Boot" Guid="{15ca44ff-4d7a-4baa-bba5-0998955e531e}"></Provider>
<EventID Qualifi

In [7]:
from azure.storage.blob import BlobServiceClient
import Evtx.Evtx as evtx
import Evtx.Views as e_views

# Function to download a file from Azure Blob Storage
def download_from_azure(storage_account_connection_string, container_name, blob_name, local_path):
    blob_service_client = BlobServiceClient.from_connection_string(storage_account_connection_string)
    container_client = blob_service_client.get_container_client(container_name)
    blob_client = container_client.get_blob_client(blob_name)
    with open(local_path, "wb") as download_file:
        download_file.write(blob_client.download_blob().readall())

# Function to display the contents of an EVTX file
def display_evtx_file_contents(file_path):
    with evtx.Evtx(file_path) as log:
        for record in log.records():
            print(record.xml())

# User-provided Azure Storage credentials and details
azure_storage_account_name = 'lemonestorageac'  # Replace with your Azure Storage account name
azure_storage_account_key = 'oCEoKyxTbTTlhupoTzWqVfeT2v5vecEo54r0PRHnG6jKCH4H8HnYtCtm0rFrGPDa9aLa9ieVpLxC+AStlakRDw=='   # Replace with your Azure Storage account key
container_name = 'cse-cic-ids2018'    # Replace with your Azure Storage container name
blob_name = 'lemonecsecicids2018out/Original Network Traffic and Log data/Friday-02-03-2018/logs/DESKTOP-AN3U28N-172.31.64.122.evtx'
azure_storage_connection_string ="DefaultEndpointsProtocol=https;AccountName=lemonestorageac;AccountKey=oCEoKyxTbTTlhupoTzWqVfeT2v5vecEo54r0PRHnG6jKCH4H8HnYtCtm0rFrGPDa9aLa9ieVpLxC+AStlakRDw==;EndpointSuffix=core.windows.net"
# Local path where the file will be downloaded
local_file_path = 'local_file(2).evtx'

# Download the .evtx file from Azure Blob Storage
download_from_azure(azure_storage_connection_string, container_name, blob_name, local_file_path)

# Display the contents of the .evtx file
display_evtx_file_contents(local_file_path)


Streaming output truncated to the last 5000 lines.
<Security UserID="S-1-5-18"></Security>
</System>
<EventData><Data Name="FinalStatus">0x00000000</Data>
<Data Name="DeviceVersionMajor">10</Data>
<Data Name="DeviceVersionMinor">0</Data>
<Data Name="DeviceNameLength">9</Data>
<Data Name="DeviceName">FileCrypt</Data>
<Data Name="DeviceTime">2015-07-09 23:14:31</Data>
<Data Name="ExtraInfoLength">197</Data>
<Data Name="ExtraInfoString">{ "flags" : "0x00000000" , "registration_version" : "0x00000203" , "tx" : false , "sections" : false , "frame" : 0 , "class_name" : "FSFilter Encryption" , "instances" : [["141100","0x00000000"]] }</Data>
<Data Name="FilterID">{02000000-0005-0000-ee9a-ad00c5afd301}</Data>
</EventData>
</Event>

<Event xmlns="http://schemas.microsoft.com/win/2004/08/events/event"><System><Provider Name="Microsoft-Windows-FilterManager" Guid="{f3c5e28e-63f6-49c7-a204-e48a1bc4b09d}"></Provider>
<EventID Qualifiers="">6</EventID>
<Version>1</Version>
<Level>4</Level>
<Task>0</

In [8]:
from azure.storage.blob import BlobServiceClient
import Evtx.Evtx as evtx
import Evtx.Views as e_views

# Function to download a file from Azure Blob Storage
def download_from_azure(storage_account_connection_string, container_name, blob_name, local_path):
    blob_service_client = BlobServiceClient.from_connection_string(storage_account_connection_string)
    container_client = blob_service_client.get_container_client(container_name)
    blob_client = container_client.get_blob_client(blob_name)
    with open(local_path, "wb") as download_file:
        download_file.write(blob_client.download_blob().readall())

# Function to display the contents of an EVTX file
def display_evtx_file_contents(file_path):
    with evtx.Evtx(file_path) as log:
        for record in log.records():
            print(record.xml())

# User-provided Azure Storage credentials and details
azure_storage_account_name = 'lemonestorageac'  # Replace with your Azure Storage account name
azure_storage_account_key = 'oCEoKyxTbTTlhupoTzWqVfeT2v5vecEo54r0PRHnG6jKCH4H8HnYtCtm0rFrGPDa9aLa9ieVpLxC+AStlakRDw=='   # Replace with your Azure Storage account key
container_name = 'cse-cic-ids2018'    # Replace with your Azure Storage container name
blob_name = 'lemonecsecicids2018out/Original Network Traffic and Log data/Friday-02-03-2018/logs/DESKTOP-AN3U28N-172.31.64.125.evtx'
azure_storage_connection_string ="DefaultEndpointsProtocol=https;AccountName=lemonestorageac;AccountKey=oCEoKyxTbTTlhupoTzWqVfeT2v5vecEo54r0PRHnG6jKCH4H8HnYtCtm0rFrGPDa9aLa9ieVpLxC+AStlakRDw==;EndpointSuffix=core.windows.net"
# Local path where the file will be downloaded
local_file_path = 'local_file(3).evtx'

# Download the .evtx file from Azure Blob Storage
download_from_azure(azure_storage_connection_string, container_name, blob_name, local_file_path)

# Display the contents of the .evtx file
display_evtx_file_contents(local_file_path)


Streaming output truncated to the last 5000 lines.
<Version>0</Version>
<Level>4</Level>
<Task>0</Task>
<Opcode>0</Opcode>
<Keywords>0x8000000000000000</Keywords>
<TimeCreated SystemTime="2018-02-27 12:17:41.456163"></TimeCreated>
<EventRecordID>3262</EventRecordID>
<Correlation ActivityID="" RelatedActivityID=""></Correlation>
<Execution ProcessID="4" ThreadID="5104"></Execution>
<Channel>System</Channel>
<Computer>DESKTOP-AN3U28N</Computer>
<Security UserID=""></Security>
</System>
<EventData><Data Name="StopTime">2018-02-27 12:17:41.456163</Data>
</EventData>
</Event>

<Event xmlns="http://schemas.microsoft.com/win/2004/08/events/event"><System><Provider Name="Microsoft-Windows-Kernel-General" Guid="{a68ca8b7-004f-d7b6-a698-07e2de0f1f5d}"></Provider>
<EventID Qualifiers="">12</EventID>
<Version>0</Version>
<Level>4</Level>
<Task>0</Task>
<Opcode>0</Opcode>
<Keywords>0x8000000000000000</Keywords>
<TimeCreated SystemTime="2018-02-27 12:18:11.686235"></TimeCreated>
<EventRecordID>3263<

In [9]:
from azure.storage.blob import BlobServiceClient
import Evtx.Evtx as evtx
import Evtx.Views as e_views

# Function to download a file from Azure Blob Storage
def download_from_azure(storage_account_connection_string, container_name, blob_name, local_path):
    blob_service_client = BlobServiceClient.from_connection_string(storage_account_connection_string)
    container_client = blob_service_client.get_container_client(container_name)
    blob_client = container_client.get_blob_client(blob_name)
    with open(local_path, "wb") as download_file:
        download_file.write(blob_client.download_blob().readall())

# Function to display the contents of an EVTX file
def display_evtx_file_contents(file_path):
    with evtx.Evtx(file_path) as log:
        for record in log.records():
            print(record.xml())

# User-provided Azure Storage credentials and details
azure_storage_account_name = 'lemonestorageac'  # Replace with your Azure Storage account name
azure_storage_account_key = 'oCEoKyxTbTTlhupoTzWqVfeT2v5vecEo54r0PRHnG6jKCH4H8HnYtCtm0rFrGPDa9aLa9ieVpLxC+AStlakRDw=='   # Replace with your Azure Storage account key
container_name = 'cse-cic-ids2018'    # Replace with your Azure Storage container name
blob_name = 'lemonecsecicids2018out/Original Network Traffic and Log data/Friday-02-03-2018/logs/DESKTOP-AN3U28N-172.31.64.126.evtx'
azure_storage_connection_string ="DefaultEndpointsProtocol=https;AccountName=lemonestorageac;AccountKey=oCEoKyxTbTTlhupoTzWqVfeT2v5vecEo54r0PRHnG6jKCH4H8HnYtCtm0rFrGPDa9aLa9ieVpLxC+AStlakRDw==;EndpointSuffix=core.windows.net"
# Local path where the file will be downloaded
local_file_path = 'local_file(4).evtx'

# Download the .evtx file from Azure Blob Storage
download_from_azure(azure_storage_connection_string, container_name, blob_name, local_file_path)

# Display the contents of the .evtx file
display_evtx_file_contents(local_file_path)


Streaming output truncated to the last 5000 lines.
<Security UserID="S-1-5-18"></Security>
</System>
<EventData><Data Name="MajorVersion">10</Data>
<Data Name="MinorVersion">0</Data>
<Data Name="BuildVersion">10240</Data>
<Data Name="QfeVersion">16384</Data>
<Data Name="ServiceVersion">0</Data>
<Data Name="BootMode">0</Data>
<Data Name="StartTime">2018-02-27 12:18:07.489586</Data>
</EventData>
</Event>

<Event xmlns="http://schemas.microsoft.com/win/2004/08/events/event"><System><Provider Name="Microsoft-Windows-Kernel-Boot" Guid="{15ca44ff-4d7a-4baa-bba5-0998955e531e}"></Provider>
<EventID Qualifiers="">20</EventID>
<Version>0</Version>
<Level>4</Level>
<Task>0</Task>
<Opcode>0</Opcode>
<Keywords>0x8000000000000000</Keywords>
<TimeCreated SystemTime="2018-02-27 12:18:07.683735"></TimeCreated>
<EventRecordID>3401</EventRecordID>
<Correlation ActivityID="" RelatedActivityID=""></Correlation>
<Execution ProcessID="4" ThreadID="8"></Execution>
<Channel>System</Channel>
<Computer>DESKTOP-A

In [11]:
from azure.storage.blob import BlobServiceClient
import Evtx.Evtx as evtx
import Evtx.Views as e_views

# Function to download a file from Azure Blob Storage
def download_from_azure(storage_account_connection_string, container_name, blob_name, local_path):
    blob_service_client = BlobServiceClient.from_connection_string(storage_account_connection_string)
    container_client = blob_service_client.get_container_client(container_name)
    blob_client = container_client.get_blob_client(blob_name)
    with open(local_path, "wb") as download_file:
        download_file.write(blob_client.download_blob().readall())

# Function to display the contents of an EVTX file
def display_evtx_file_contents(file_path):
    with evtx.Evtx(file_path) as log:
        for record in log.records():
            print(record.xml())

# User-provided Azure Storage credentials and details
azure_storage_account_name = 'lemonestorageac'  # Replace with your Azure Storage account name
azure_storage_account_key = 'oCEoKyxTbTTlhupoTzWqVfeT2v5vecEo54r0PRHnG6jKCH4H8HnYtCtm0rFrGPDa9aLa9ieVpLxC+AStlakRDw=='   # Replace with your Azure Storage account key
container_name = 'cse-cic-ids2018'    # Replace with your Azure Storage container name
blob_name = 'lemonecsecicids2018out/Original Network Traffic and Log data/Friday-16-02-2018/logs.zip'
azure_storage_connection_string ="DefaultEndpointsProtocol=https;AccountName=lemonestorageac;AccountKey=oCEoKyxTbTTlhupoTzWqVfeT2v5vecEo54r0PRHnG6jKCH4H8HnYtCtm0rFrGPDa9aLa9ieVpLxC+AStlakRDw==;EndpointSuffix=core.windows.net"
# Local path where the file will be downloaded
local_file_path = 'local_file(5).evtx'

# Download the .evtx file from Azure Blob Storage
download_from_azure(azure_storage_connection_string, container_name, blob_name, local_file_path)

# Display the contents of the .evtx file
display_evtx_file_contents(local_file_path)


In [12]:
from azure.storage.blob import BlobServiceClient
import Evtx.Evtx as evtx
import Evtx.Views as e_views

# Function to download a file from Azure Blob Storage
def download_from_azure(storage_account_connection_string, container_name, blob_name, local_path):
    blob_service_client = BlobServiceClient.from_connection_string(storage_account_connection_string)
    container_client = blob_service_client.get_container_client(container_name)
    blob_client = container_client.get_blob_client(blob_name)
    with open(local_path, "wb") as download_file:
        download_file.write(blob_client.download_blob().readall())

# Function to display the contents of an EVTX file
def display_evtx_file_contents(file_path):
    with evtx.Evtx(file_path) as log:
        for record in log.records():
            print(record.xml())

# User-provided Azure Storage credentials and details
azure_storage_account_name = 'lemonestorageac'  # Replace with your Azure Storage account name
azure_storage_account_key = 'oCEoKyxTbTTlhupoTzWqVfeT2v5vecEo54r0PRHnG6jKCH4H8HnYtCtm0rFrGPDa9aLa9ieVpLxC+AStlakRDw=='   # Replace with your Azure Storage account key
container_name = 'cse-cic-ids2018'    # Replace with your Azure Storage container name
blob_name = 'lemonecsecicids2018out/Original Network Traffic and Log data/Friday-02-03-2018/logs/DESKTOP-AN3U28N-172.31.64.26.evtx'
azure_storage_connection_string ="DefaultEndpointsProtocol=https;AccountName=lemonestorageac;AccountKey=oCEoKyxTbTTlhupoTzWqVfeT2v5vecEo54r0PRHnG6jKCH4H8HnYtCtm0rFrGPDa9aLa9ieVpLxC+AStlakRDw==;EndpointSuffix=core.windows.net"
# Local path where the file will be downloaded
local_file_path = 'local_file(6).evtx'

# Download the .evtx file from Azure Blob Storage
download_from_azure(azure_storage_connection_string, container_name, blob_name, local_file_path)

# Display the contents of the .evtx file
display_evtx_file_contents(local_file_path)


Streaming output truncated to the last 5000 lines.
<Channel>System</Channel>
<Computer>DESKTOP-AN3U28N</Computer>
<Security UserID="S-1-5-18"></Security>
</System>
<EventData><Data Name="BootType">0</Data>
</EventData>
</Event>

<Event xmlns="http://schemas.microsoft.com/win/2004/08/events/event"><System><Provider Name="Microsoft-Windows-Kernel-Boot" Guid="{15ca44ff-4d7a-4baa-bba5-0998955e531e}"></Provider>
<EventID Qualifiers="">25</EventID>
<Version>0</Version>
<Level>4</Level>
<Task>0</Task>
<Opcode>0</Opcode>
<Keywords>0x8000000000000000</Keywords>
<TimeCreated SystemTime="2018-02-27 12:17:50.681847"></TimeCreated>
<EventRecordID>3264</EventRecordID>
<Correlation ActivityID="" RelatedActivityID=""></Correlation>
<Execution ProcessID="4" ThreadID="8"></Execution>
<Channel>System</Channel>
<Computer>DESKTOP-AN3U28N</Computer>
<Security UserID="S-1-5-18"></Security>
</System>
<EventData><Data Name="BootMenuPolicy">1</Data>
</EventData>
</Event>

<Event xmlns="http://schemas.microsoft.

In [13]:
from azure.storage.blob import BlobServiceClient
import Evtx.Evtx as evtx
import Evtx.Views as e_views

# Function to download a file from Azure Blob Storage
def download_from_azure(storage_account_connection_string, container_name, blob_name, local_path):
    blob_service_client = BlobServiceClient.from_connection_string(storage_account_connection_string)
    container_client = blob_service_client.get_container_client(container_name)
    blob_client = container_client.get_blob_client(blob_name)
    with open(local_path, "wb") as download_file:
        download_file.write(blob_client.download_blob().readall())

# Function to display the contents of an EVTX file
def display_evtx_file_contents(file_path):
    with evtx.Evtx(file_path) as log:
        for record in log.records():
            print(record.xml())

# User-provided Azure Storage credentials and details
azure_storage_account_name = 'lemonestorageac'  # Replace with your Azure Storage account name
azure_storage_account_key = 'oCEoKyxTbTTlhupoTzWqVfeT2v5vecEo54r0PRHnG6jKCH4H8HnYtCtm0rFrGPDa9aLa9ieVpLxC+AStlakRDw=='   # Replace with your Azure Storage account key
container_name = 'cse-cic-ids2018'    # Replace with your Azure Storage container name
blob_name = 'lemonecsecicids2018out/Original Network Traffic and Log data/Friday-02-03-2018/logs/DESKTOP-AN3U28N-172.31.64.35.evtx'
azure_storage_connection_string ="DefaultEndpointsProtocol=https;AccountName=lemonestorageac;AccountKey=oCEoKyxTbTTlhupoTzWqVfeT2v5vecEo54r0PRHnG6jKCH4H8HnYtCtm0rFrGPDa9aLa9ieVpLxC+AStlakRDw==;EndpointSuffix=core.windows.net"
# Local path where the file will be downloaded
local_file_path = 'local_file(7).evtx'

# Download the .evtx file from Azure Blob Storage
download_from_azure(azure_storage_connection_string, container_name, blob_name, local_file_path)

# Display the contents of the .evtx file
display_evtx_file_contents(local_file_path)


Streaming output truncated to the last 5000 lines.
<Channel>System</Channel>
<Computer>DESKTOP-AN3U28N</Computer>
<Security UserID="S-1-5-21-913095808-1976597276-3100954228-500"></Security>
</System>
<EventData><Data Name="HiveNameLength">184</Data>
<Data Name="HiveName">\??\C:\Users\Administrator\AppData\Local\Packages\Microsoft.Advertising.Xaml_8wekyb3d8bbwe\Microsoft.Advertising.Xaml_10.1802.1.0_x64__8wekyb3d8bbwe\ActivationStore\ActivationStore.dat</Data>
<Data Name="KeysUpdated">0</Data>
<Data Name="DirtyPages">0</Data>
</EventData>
</Event>

<Event xmlns="http://schemas.microsoft.com/win/2004/08/events/event"><System><Provider Name="Microsoft-Windows-WindowsUpdateClient" Guid="{945a8954-c147-4acd-923f-40c45405a658}"></Provider>
<EventID Qualifiers="">44</EventID>
<Version>1</Version>
<Level>4</Level>
<Task>1</Task>
<Opcode>12</Opcode>
<Keywords>0x8000000000002004</Keywords>
<TimeCreated SystemTime="2018-02-28 03:16:20.308731"></TimeCreated>
<EventRecordID>3355</EventRecordID>
<Co

In [14]:
from azure.storage.blob import BlobServiceClient
import Evtx.Evtx as evtx
import Evtx.Views as e_views

# Function to download a file from Azure Blob Storage
def download_from_azure(storage_account_connection_string, container_name, blob_name, local_path):
    blob_service_client = BlobServiceClient.from_connection_string(storage_account_connection_string)
    container_client = blob_service_client.get_container_client(container_name)
    blob_client = container_client.get_blob_client(blob_name)
    with open(local_path, "wb") as download_file:
        download_file.write(blob_client.download_blob().readall())

# Function to display the contents of an EVTX file
def display_evtx_file_contents(file_path):
    with evtx.Evtx(file_path) as log:
        for record in log.records():
            print(record.xml())

# User-provided Azure Storage credentials and details
azure_storage_account_name = 'lemonestorageac'  # Replace with your Azure Storage account name
azure_storage_account_key = 'oCEoKyxTbTTlhupoTzWqVfeT2v5vecEo54r0PRHnG6jKCH4H8HnYtCtm0rFrGPDa9aLa9ieVpLxC+AStlakRDw=='   # Replace with your Azure Storage account key
container_name = 'cse-cic-ids2018'    # Replace with your Azure Storage container name
blob_name = 'lemonecsecicids2018out/Original Network Traffic and Log data/Friday-02-03-2018/logs/DESKTOP-AN3U28N-172.31.64.38.evtx'
azure_storage_connection_string ="DefaultEndpointsProtocol=https;AccountName=lemonestorageac;AccountKey=oCEoKyxTbTTlhupoTzWqVfeT2v5vecEo54r0PRHnG6jKCH4H8HnYtCtm0rFrGPDa9aLa9ieVpLxC+AStlakRDw==;EndpointSuffix=core.windows.net"
# Local path where the file will be downloaded
local_file_path = 'local_file(8).evtx'

# Download the .evtx file from Azure Blob Storage
download_from_azure(azure_storage_connection_string, container_name, blob_name, local_file_path)

# Display the contents of the .evtx file
display_evtx_file_contents(local_file_path)


Streaming output truncated to the last 5000 lines.
<TimeCreated SystemTime="2018-02-27 12:18:04.698792"></TimeCreated>
<EventRecordID>3263</EventRecordID>
<Correlation ActivityID="" RelatedActivityID=""></Correlation>
<Execution ProcessID="4" ThreadID="8"></Execution>
<Channel>System</Channel>
<Computer>DESKTOP-AN3U28N</Computer>
<Security UserID="S-1-5-18"></Security>
</System>
<EventData><Data Name="FinalStatus">0x00000000</Data>
<Data Name="DeviceVersionMajor">10</Data>
<Data Name="DeviceVersionMinor">0</Data>
<Data Name="DeviceNameLength">8</Data>
<Data Name="DeviceName">WdFilter</Data>
<Data Name="DeviceTime">2015-07-09 23:19:05</Data>
<Data Name="ExtraInfoLength">196</Data>
<Data Name="ExtraInfoString">{ "flags" : "0x00000010" , "registration_version" : "0x00000203" , "tx" : true , "sections" : false , "frame" : 0 , "class_name" : "FSFilter Anti-Virus" , "instances" : [["328010","0x00000000"]] }</Data>
<Data Name="FilterID">{02000000-0003-0000-9ca8-bf04c5afd301}</Data>
</EventDat

In [15]:
from azure.storage.blob import BlobServiceClient
import Evtx.Evtx as evtx
import Evtx.Views as e_views

# Function to download a file from Azure Blob Storage
def download_from_azure(storage_account_connection_string, container_name, blob_name, local_path):
    blob_service_client = BlobServiceClient.from_connection_string(storage_account_connection_string)
    container_client = blob_service_client.get_container_client(container_name)
    blob_client = container_client.get_blob_client(blob_name)
    with open(local_path, "wb") as download_file:
        download_file.write(blob_client.download_blob().readall())

# Function to display the contents of an EVTX file
def display_evtx_file_contents(file_path):
    with evtx.Evtx(file_path) as log:
        for record in log.records():
            print(record.xml())

# User-provided Azure Storage credentials and details
azure_storage_account_name = 'lemonestorageac'  # Replace with your Azure Storage account name
azure_storage_account_key = 'oCEoKyxTbTTlhupoTzWqVfeT2v5vecEo54r0PRHnG6jKCH4H8HnYtCtm0rFrGPDa9aLa9ieVpLxC+AStlakRDw=='   # Replace with your Azure Storage account key
container_name = 'cse-cic-ids2018'    # Replace with your Azure Storage container name
blob_name = 'lemonecsecicids2018out/Original Network Traffic and Log data/Friday-02-03-2018/logs/DESKTOP-AN3U28N-172.31.64.46.evtx'
azure_storage_connection_string ="DefaultEndpointsProtocol=https;AccountName=lemonestorageac;AccountKey=oCEoKyxTbTTlhupoTzWqVfeT2v5vecEo54r0PRHnG6jKCH4H8HnYtCtm0rFrGPDa9aLa9ieVpLxC+AStlakRDw==;EndpointSuffix=core.windows.net"
# Local path where the file will be downloaded
local_file_path = 'local_file(9).evtx'

# Download the .evtx file from Azure Blob Storage
download_from_azure(azure_storage_connection_string, container_name, blob_name, local_file_path)

# Display the contents of the .evtx file
display_evtx_file_contents(local_file_path)


Streaming output truncated to the last 5000 lines.
<Execution ProcessID="652" ThreadID="968"></Execution>
<Channel>System</Channel>
<Computer>DESKTOP-AN3U28N</Computer>
<Security UserID="S-1-5-18"></Security>
</System>
<EventData><Data Name="TSId">1</Data>
<Data Name="UserSid">S-1-5-21-913095808-1976597276-3100954228-500</Data>
</EventData>
</Event>

<Event xmlns="http://schemas.microsoft.com/win/2004/08/events/event"><System><Provider Name="Microsoft-Windows-DHCPv6-Client" Guid="{6a1f2b00-6a90-4c38-95a5-5cab3b056778}"></Provider>
<EventID Qualifiers="">51047</EventID>
<Version>0</Version>
<Level>4</Level>
<Task>4</Task>
<Opcode>63</Opcode>
<Keywords>0x2000000000000000</Keywords>
<TimeCreated SystemTime="2018-02-27 12:17:19.058903"></TimeCreated>
<EventRecordID>3244</EventRecordID>
<Correlation ActivityID="" RelatedActivityID=""></Correlation>
<Execution ProcessID="388" ThreadID="1308"></Execution>
<Channel>System</Channel>
<Computer>DESKTOP-AN3U28N</Computer>
<Security UserID="S-1-5-1

In [16]:
from azure.storage.blob import BlobServiceClient
import Evtx.Evtx as evtx
import Evtx.Views as e_views

# Function to download a file from Azure Blob Storage
def download_from_azure(storage_account_connection_string, container_name, blob_name, local_path):
    blob_service_client = BlobServiceClient.from_connection_string(storage_account_connection_string)
    container_client = blob_service_client.get_container_client(container_name)
    blob_client = container_client.get_blob_client(blob_name)
    with open(local_path, "wb") as download_file:
        download_file.write(blob_client.download_blob().readall())

# Function to display the contents of an EVTX file
def display_evtx_file_contents(file_path):
    with evtx.Evtx(file_path) as log:
        for record in log.records():
            print(record.xml())

# User-provided Azure Storage credentials and details
azure_storage_account_name = 'lemonestorageac'  # Replace with your Azure Storage account name
azure_storage_account_key = 'oCEoKyxTbTTlhupoTzWqVfeT2v5vecEo54r0PRHnG6jKCH4H8HnYtCtm0rFrGPDa9aLa9ieVpLxC+AStlakRDw=='   # Replace with your Azure Storage account key
container_name = 'cse-cic-ids2018'    # Replace with your Azure Storage container name
blob_name = 'lemonecsecicids2018out/Original Network Traffic and Log data/Friday-02-03-2018/logs/DESKTOP-AN3U28N-172.31.64.54.evtx'
azure_storage_connection_string ="DefaultEndpointsProtocol=https;AccountName=lemonestorageac;AccountKey=oCEoKyxTbTTlhupoTzWqVfeT2v5vecEo54r0PRHnG6jKCH4H8HnYtCtm0rFrGPDa9aLa9ieVpLxC+AStlakRDw==;EndpointSuffix=core.windows.net"
# Local path where the file will be downloaded
local_file_path = 'local_file(10).evtx'

# Download the .evtx file from Azure Blob Storage
download_from_azure(azure_storage_connection_string, container_name, blob_name, local_file_path)

# Display the contents of the .evtx file
display_evtx_file_contents(local_file_path)


Streaming output truncated to the last 5000 lines.
<Channel>System</Channel>
<Computer>DESKTOP-AN3U28N</Computer>
<Security UserID="S-1-5-18"></Security>
</System>
<EventData><Data Name="LastShutdownGood">True</Data>
<Data Name="LastBootGood">True</Data>
</EventData>
</Event>

<Event xmlns="http://schemas.microsoft.com/win/2004/08/events/event"><System><Provider Name="Microsoft-Windows-Kernel-Boot" Guid="{15ca44ff-4d7a-4baa-bba5-0998955e531e}"></Provider>
<EventID Qualifiers="">27</EventID>
<Version>0</Version>
<Level>4</Level>
<Task>0</Task>
<Opcode>0</Opcode>
<Keywords>0x8000000000000000</Keywords>
<TimeCreated SystemTime="2018-02-27 12:17:53.687416"></TimeCreated>
<EventRecordID>3288</EventRecordID>
<Correlation ActivityID="" RelatedActivityID=""></Correlation>
<Execution ProcessID="4" ThreadID="8"></Execution>
<Channel>System</Channel>
<Computer>DESKTOP-AN3U28N</Computer>
<Security UserID="S-1-5-18"></Security>
</System>
<EventData><Data Name="BootType">0</Data>
</EventData>
</Even

In [17]:
from azure.storage.blob import BlobServiceClient
import Evtx.Evtx as evtx
import Evtx.Views as e_views

# Function to download a file from Azure Blob Storage
def download_from_azure(storage_account_connection_string, container_name, blob_name, local_path):
    blob_service_client = BlobServiceClient.from_connection_string(storage_account_connection_string)
    container_client = blob_service_client.get_container_client(container_name)
    blob_client = container_client.get_blob_client(blob_name)
    with open(local_path, "wb") as download_file:
        download_file.write(blob_client.download_blob().readall())

# Function to display the contents of an EVTX file
def display_evtx_file_contents(file_path):
    with evtx.Evtx(file_path) as log:
        for record in log.records():
            print(record.xml())

# User-provided Azure Storage credentials and details
azure_storage_account_name = 'lemonestorageac'  # Replace with your Azure Storage account name
azure_storage_account_key = 'oCEoKyxTbTTlhupoTzWqVfeT2v5vecEo54r0PRHnG6jKCH4H8HnYtCtm0rFrGPDa9aLa9ieVpLxC+AStlakRDw=='   # Replace with your Azure Storage account key
container_name = 'cse-cic-ids2018'    # Replace with your Azure Storage container name
blob_name = 'lemonecsecicids2018out/Original Network Traffic and Log data/Friday-02-03-2018/logs/DESKTOP-AN3U28N-172.31.64.63.evtx'
azure_storage_connection_string ="DefaultEndpointsProtocol=https;AccountName=lemonestorageac;AccountKey=oCEoKyxTbTTlhupoTzWqVfeT2v5vecEo54r0PRHnG6jKCH4H8HnYtCtm0rFrGPDa9aLa9ieVpLxC+AStlakRDw==;EndpointSuffix=core.windows.net"
# Local path where the file will be downloaded
local_file_path = 'local_file(11).evtx'

# Download the .evtx file from Azure Blob Storage
download_from_azure(azure_storage_connection_string, container_name, blob_name, local_file_path)

# Display the contents of the .evtx file
display_evtx_file_contents(local_file_path)


Streaming output truncated to the last 5000 lines.
</EventData>
</Event>

<Event xmlns="http://schemas.microsoft.com/win/2004/08/events/event"><System><Provider Name="Microsoft-Windows-Kernel-Boot" Guid="{15ca44ff-4d7a-4baa-bba5-0998955e531e}"></Provider>
<EventID Qualifiers="">32</EventID>
<Version>0</Version>
<Level>4</Level>
<Task>0</Task>
<Opcode>0</Opcode>
<Keywords>0x8000000000000000</Keywords>
<TimeCreated SystemTime="2018-02-27 12:17:48.682636"></TimeCreated>
<EventRecordID>3299</EventRecordID>
<Correlation ActivityID="" RelatedActivityID=""></Correlation>
<Execution ProcessID="4" ThreadID="8"></Execution>
<Channel>System</Channel>
<Computer>DESKTOP-AN3U28N</Computer>
<Security UserID="S-1-5-18"></Security>
</System>
<EventData><Data Name="BitlockerUserInputTime">0</Data>
</EventData>
</Event>

<Event xmlns="http://schemas.microsoft.com/win/2004/08/events/event"><System><Provider Name="Microsoft-Windows-FilterManager" Guid="{f3c5e28e-63f6-49c7-a204-e48a1bc4b09d}"></Provider>
<E

In [35]:
from azure.storage.blob import BlobServiceClient
import os
import xml.etree.ElementTree as ET
import Evtx.Evtx as evtx
import Evtx.Views as e_views

# Function to download a file from Azure Blob Storage
def download_from_azure(storage_account_connection_string, container_name, blob_name, local_path):
    blob_service_client = BlobServiceClient.from_connection_string(storage_account_connection_string)
    container_client = blob_service_client.get_container_client(container_name)
    blob_client = container_client.get_blob_client(blob_name)
    with open(local_path, "wb") as download_file:
        download_file.write(blob_client.download_blob().readall())

# Function to convert EVTX file to XML without evtx_dump
def convert_evtx_to_xml(evtx_file_path, xml_output_path):
    with evtx.Evtx(evtx_file_path) as log:
        root = ET.Element("Events")
        for record in log.records():
            event_xml = record.xml()
            event_element = ET.fromstring(event_xml)
            root.append(event_element)

        tree = ET.ElementTree(root)
        tree.write(xml_output_path)

# Function to display the contents of an XML file
def display_xml_file_contents(xml_file_path):
    tree = ET.parse(xml_file_path)
    root = tree.getroot()
    for event_element in root:
        event_xml = ET.tostring(event_element, encoding="unicode")
        print(event_xml)



# User-provided Azure Storage credentials and details
azure_storage_connection_string = 'DefaultEndpointsProtocol=https;AccountName=lemonestorageac;AccountKey=oCEoKyxTbTTlhupoTzWqVfeT2v5vecEo54r0PRHnG6jKCH4H8HnYtCtm0rFrGPDa9aLa9ieVpLxC+AStlakRDw==;EndpointSuffix=core.windows.net'  # Replace with your Azure Storage connection string
container_name = 'cse-cic-ids2018'        # Replace with your Azure Storage container name
blob_name = 'lemonecsecicids2018out/Original Network Traffic and Log data/Friday-02-03-2018/logs/DESKTOP-AN3U28N-172.31.64.63.evtx'



# Local path where the EVTX file will be downloaded
local_evtx_file_path = '/content/local_file(11).evtx'

# Local path where the XML file will be saved
local_xml_file_path = 'local_file.xml'

# Download the EVTX file from Azure Blob Storage
download_from_azure(azure_storage_connection_string, container_name, blob_name, local_evtx_file_path)

# Convert the EVTX file to XML without evtx_dump
convert_evtx_to_xml(local_evtx_file_path, local_xml_file_path)

# Display the contents of the XML file
display_xml_file_contents(local_xml_file_path)


Streaming output truncated to the last 5000 lines.
<ns0:EventData><ns0:Data Name="MajorVersion">10</ns0:Data>
<ns0:Data Name="MinorVersion">0</ns0:Data>
<ns0:Data Name="BuildVersion">10240</ns0:Data>
<ns0:Data Name="QfeVersion">16384</ns0:Data>
<ns0:Data Name="ServiceVersion">0</ns0:Data>
<ns0:Data Name="BootMode">0</ns0:Data>
<ns0:Data Name="StartTime">2018-02-10 13:21:14.494272</ns0:Data>
</ns0:EventData>
</ns0:Event>
<ns0:Event xmlns:ns0="http://schemas.microsoft.com/win/2004/08/events/event"><ns0:System><ns0:Provider Name="Microsoft-Windows-Kernel-Boot" Guid="{15ca44ff-4d7a-4baa-bba5-0998955e531e}" />
<ns0:EventID Qualifiers="">20</ns0:EventID>
<ns0:Version>0</ns0:Version>
<ns0:Level>4</ns0:Level>
<ns0:Task>0</ns0:Task>
<ns0:Opcode>0</ns0:Opcode>
<ns0:Keywords>0x8000000000000000</ns0:Keywords>
<ns0:TimeCreated SystemTime="2018-02-10 13:21:14.686020" />
<ns0:EventRecordID>2481</ns0:EventRecordID>
<ns0:Correlation ActivityID="" RelatedActivityID="" />
<ns0:Execution ProcessID="4" Thr

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [37]:
import csv

# Function to convert XML file to CSV
def convert_xml_to_csv(xml_file_path, csv_output_path):
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    # Open a CSV file for writing
    with open(csv_output_path, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)

        # Write header row based on the XML structure
        header = []
        for event_element in root:
            for field in event_element:
                if field.tag not in header:
                    header.append(field.tag)
        csv_writer.writerow(header)

        # Write data rows
        for event_element in root:
            row = []
            for field_tag in header:
                field_value = event_element.find(field_tag)
                if field_value is not None:
                    row.append(field_value.text)
                else:
                    row.append("")  # Handle missing values
            csv_writer.writerow(row)

local_xml_file_path = '/content/local_file.xml'

# User-provided CSV file path
local_csv_file_path = 'log_file.csv'

# Convert the XML file to CSV
convert_xml_to_csv(local_xml_file_path, local_csv_file_path)


In [33]:
from azure.storage.blob import BlobServiceClient
import csv
import os
import Evtx.Evtx as evtx
import Evtx.Views as e_views

# Function to download a file from Azure Blob Storage
def download_from_azure(storage_account_connection_string, container_name, blob_name, local_path):
    blob_service_client = BlobServiceClient.from_connection_string(storage_account_connection_string)
    container_client = blob_service_client.get_container_client(container_name)
    blob_client = container_client.get_blob_client(blob_name)
    with open(local_path, "wb") as download_file:
        download_file.write(blob_client.download_blob().readall())

import xml.etree.ElementTree as ET

# Function to convert EVTX file to CSV
def convert_evtx_to_csv(evtx_file_path, csv_output_path):
    with evtx.Evtx(evtx_file_path) as log:
        with open(csv_output_path, 'w', newline='', encoding='utf-8') as csv_file:
            csv_writer = csv.writer(csv_file)

            # Write header row with column names
            csv_writer.writerow(['EventID', 'TimeCreated', 'ProviderName', 'ComputerName', 'Message'])

            for record in log.records():
                # Extract relevant information from the EVTX record
                xml_string = record.xml()
                root_element = ET.fromstring(xml_string)

                # Access TimeCreated element
                time_created_element = root_element.find(".//TimeCreated SystemTime")
                time_created = time_created_element.get("SystemTime") if time_created_element is not None else None

                # Access EventID element
                event_id_element = root_element.find(".//EventID Qualifiers")
                event_id = event_id_element.text if event_id_element is not None else None

                # Access ProviderName element
                provider_element = root_element.find(".//Provider Name")
                provider_name = provider_element.get("Name") if provider_element is not None else None

                # Access Computer element
                computer_element = root_element.find(".//Computer")
                computer_name = computer_element.text if computer_element is not None else None

                # Extracting details from EventData for message (adjust as needed)
                event_data_element = root_element.find(".//EventData")
                message = ET.tostring(event_data_element, encoding="unicode") if event_data_element is not None else None

                # Write the extracted information to the CSV file
                csv_writer.writerow([event_id, time_created, provider_name, computer_name, message])


# User-provided Azure Storage credentials and details
azure_storage_connection_string = 'DefaultEndpointsProtocol=https;AccountName=lemonestorageac;AccountKey=oCEoKyxTbTTlhupoTzWqVfeT2v5vecEo54r0PRHnG6jKCH4H8HnYtCtm0rFrGPDa9aLa9ieVpLxC+AStlakRDw==;EndpointSuffix=core.windows.net'  # Replace with your Azure Storage connection string
container_name = 'cse-cic-ids2018'        # Replace with your Azure Storage container name
blob_name = 'lemonecsecicids2018out/Original Network Traffic and Log data/Friday-02-03-2018/logs/DESKTOP-AN3U28N-172.31.64.63.evtx'

# Local paths for the downloaded EVTX file and converted CSV file
local_evtx_file_path = '/content/local_file(11).evtx'
local_csv_file_path = 'local_file.csv'

# Download the .evtx file from Azure Blob Storage
download_from_azure(azure_storage_connection_string, container_name, blob_name, local_evtx_file_path)

# Convert EVTX to CSV
convert_evtx_to_csv(local_evtx_file_path, local_csv_file_path)
